# Set up

In [352]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install litellm --quiet
!pip install deep-translator --quiet

In [305]:
from litellm import completion
from deep_translator import GoogleTranslator
import os
import math
from nltk import CFG
from nltk.parse.generate import generate
import itertools
import random
import ast

## set ENV variables
os.environ["OPENAI_API_KEY"] = ""

In [3]:
gpt4 = False

m = "gpt-4"
if not gpt4:
  m = "gpt-3.5-turbo"

In [148]:
fruits = ['Apple', 'Apricot', 'Avocado', 'Banana', 'Blackberry', 'Blackcurrant', 'Blueberry', 'Cherry', 'Coconut', 'Date', 'Dragonfruit', 'Durian', 'Elderberry', 'Fig', 'Grape', 'Grapefruit', 'Guava', 'Jackfruit', 'Kiwifruit', 'Lemon', 'Lime', 'Lychee', 'Mango', 'Melon', 'Nectarine', 'Orange', 'Papaya', 'Passionfruit', 'Peach', 'Pear']

#helper functions

In [372]:
import numpy as np

predicted = [1, 0, 0, 0]
expected = [1, 1, 0, 0]

def confusion_matrix(predicted, expected):
  tp = sum(p*e for p, e in zip(predicted, expected)) # True positives
  fp = sum(p*(1-e) for p, e in zip(predicted, expected)) # False positives
  fn = sum((1-p)*e for p, e in zip(predicted, expected)) # False negatives
  tn = sum((1-p)*(1-e) for p, e in zip(predicted, expected)) # True negatives

  precision = tp / (tp + fp)
  recall = tp / (tp + fn)
  f1 = 0
  if tp != 0:
    f1 = (2*precision*recall) / (precision+recall)
  accuracy = (tp + tn)/ (tp + tn + fp + fn)
  return {"precision": precision, "recall":recall, "f1":f1, "accuracy":accuracy}

confusion = confusion_matrix(predicted, expected)
print("F1 Score: ", confusion['f1'])
print("Recall: ", confusion['recall'])
print("Precision: ", confusion["precision"])
print("Accuracy: ", confusion["accuracy"])

F1 Score:  0.6666666666666666
Recall:  0.5
Precision:  1.0
Accuracy:  0.75


In [4]:
def answer(q):
  messages = [{"content": q,"role": "user"}]
  response = completion(model=m, messages=messages)
  return response.choices[0].message.content

In [5]:
def translate(text, from_ ="uk", to="en"):
    to_translate = text
    translated = GoogleTranslator(source=from_, target=to).translate(to_translate)
    return translated
translate("ти чи він", from_ ="uk", to="en")

'you or him'

In [133]:
def count_nouns(s: str):
  """
  Count 'N's in a string.
  "N or N and N" -> 3
  """
  counter = 0
  for t in s:
    if t == "N":
      counter+=1
  return counter

In [311]:
def dot_product(vec1, vec2):
    return sum(x * y for x, y in zip(vec1, vec2))

def magnitude(vec):
    return math.sqrt(sum(x**2 for x in vec))

def cosine(vec1, vec2):
    dot_prod = dot_product(vec1, vec2)
    mag1 = magnitude(vec1)
    mag2 = magnitude(vec2)

    if mag1 == 0 or mag2 == 0:
        return 0  # Avoid division by zero
    return dot_prod / (mag1 * mag2)

def coincidence(v1, v2):
  assert len(v1) == len(v2)
  counter = 0
  for i in range(len(v1)):
    counter += int(v1[i] == v2[i])
  return float(counter/len(v1))

In [160]:
cosine([0, 1, 1, 1], [0, 0, 0, 1])

0.5773502691896258

## 4-way base (dnf, cnf, left-first, right-first)

In [150]:
#cnf
expr = "N and N or N and N or N or N and N"

def four_parenthesize(expr: str):
  base = {"dnf":expr, #python default interpretation is DNF
          "cnf":"",
          "left":"",
          "right":""
          }
  #cnf break
  exp = expr.split("and")
  res = ""
  for i in range(len(exp)):
    res += " ( "
    res += exp[i]
    res += " ) "
    if i != len(exp) - 1:
      res += "and"
  base['cnf'] = res

  #left_first break
  exp = expr.split()
  l = len(exp)
  res = ""
  if l < 3:
    base['left'] = expr
  else:
    for j in range(int((l-1)/2)):
      res+=" ( "
    i = 0
    res += exp[i]
    res += " "
    while i < l-2:
      res+=exp[i+1]
      res+=" "
      res+=exp[i+2]
      res+=" ) "
      i+=2
    base['left'] = res

  #right generation
  exp = expr.split()
  l = len(exp)
  res = ""
  if l <= 3:
    base['right'] = expr
  else:
      res += exp[0]
      res += " "
      res += exp[1]
      i = 2
      res += " "
      while i < l-1:
        res += " ( "
        res+=exp[i]
        res+=" "
        res+=exp[i+1]
        res+=" "
        i+=2
      res += exp[i]
      for j in range(int((l-1)/2) - 1):
        res += " ) "
      base['right'] = res
  return base

In [152]:
four_parenthesize(expr)

{'dnf': 'N and N or N and N or N or N and N',
 'cnf': ' ( N  ) and (  N or N  ) and (  N or N or N  ) and (  N ) ',
 'left': ' (  (  (  (  (  ( N and N ) or N ) and N ) or N ) or N ) and N ) ',
 'right': 'N and  ( N or  ( N and  ( N or  ( N or  ( N and N )  )  )  )  ) '}

# Generate query

In [340]:
# generate a string from the BNF
def gen_grammar_sequences(grammar, d=5):
    #With this we "create" all the possible combinations
  grammar.productions()

  #Here you can see all the productions (sentences) with 5 words
  #created with this grammar
  possible_sequences = []
  i = 0
  for production in generate(grammar, depth = d):
    possible_sequences.append(''.join(production))
    i += 1
  possible_sequences = list(set(possible_sequences))
  # for p in possible_sequences:
  #   print(p)
  return(possible_sequences)

bool_expressions = CFG.fromstring("""
OrP -> AndP | AndP " or " OrP
AndP -> "N" | "N and " AndP
 """)
(gen_grammar_sequences(bool_expressions, d = 6))

['N or N',
 'N and N or N and N and N or N',
 'N or N or N or N',
 'N and N and N and N or N and N and N or N',
 'N or N and N or N and N',
 'N and N or N or N or N',
 'N and N and N or N and N and N or N or N',
 'N and N and N and N or N and N or N',
 'N and N and N or N and N or N',
 'N or N and N and N',
 'N and N and N and N or N or N and N or N',
 'N and N and N or N and N and N or N and N or N',
 'N or N or N',
 'N and N and N or N and N and N or N',
 'N and N or N and N or N or N',
 'N or N or N and N',
 'N and N and N or N',
 'N and N and N or N and N',
 'N and N or N and N or N',
 'N and N and N and N or N and N or N or N',
 'N and N and N and N or N and N and N or N or N',
 'N and N and N and N or N and N and N or N and N or N',
 'N or N and N or N or N',
 'N and N or N and N and N',
 'N and N and N or N or N or N',
 'N',
 'N and N and N or N and N or N and N',
 'N or N and N and N or N and N or N',
 'N and N and N or N or N and N or N',
 'N and N or N',
 'N and N or N and N 

In [341]:
def eval_expression(expr, values):
    # Evaluates a simple boolean expression (without nested parentheses)
    for var, val in values.items():
        expr = expr.replace(var, str(val))
    return eval(expr)

# Example usage
expr = "(bag or cat) and (not bag or cat)"
values = {'apple': True, 'bag': True, 'cat':False}
result = eval_expression(expr, values)
print(result)  # Expected output: True

False


In [342]:
def replace_nouns(s, nouns):
  """
  ("N or N", ["bag", "cat"]) -> "bag or cat"
  """
  ns = (nouns).copy()
  assert count_nouns(s) == len(nouns)
  s = s.split()
  for i in range(len(s)):
    if s[i] == "N":
      s[i] = ns.pop()
  return " ".join(s)

In [343]:
def gen_all_combos(items):
  """
  ["cat"] -> [cat, not cat]
  """
  def negate(noun, neg=False):
    if not neg:
      return f"not {noun}"
    return noun

  n = len(items)
  combinations = list(itertools.product([1, 0], repeat=n))
  res = [[negate(items[i], neg=c[i]) for i in range(len(c))] for c in combinations]
  return res

In [344]:
def delete_neg(nouns):
  def not_term(term):
    return "not" in term
  res = [k for k in nouns if not not_term(k)]
  if not res:
    return ["nothing"]
  return res

# null hypothesis

In [345]:
# Generate all combinations of True and False for n variables
def find_vector(expr: str, nouns: list):
  combinations = gen_all_combos(nouns)
  vector = []
  # Create the truth table
  for combo in combinations:
      e = replace_nouns(expr, combo)
      res = eval_expression(e, dict(zip(nouns, len(nouns)*[True])))
      vector.append(int(res))
  return vector[::-1]

In [346]:
find_vector("( N or N ) and ( N or N )", ["bag", "apple", "cat", "dog"])

[0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1]

In [370]:
def answer_qset(q, nouns):
  res = []
  q = f'Andre asked Boris: "Please buy me: {q}".\
  Boris bought {", ".join(nouns)}. Did Boris meet Andre\'s request? Oversatisfaction is allowed. Explain and write: 1 if "yes", 0 otherwise. write a number'
  a = answer(q)
  i = int("1" in a)
  res.append(i)
  return {"res":res, "q": q, "a":a}

# test

In [266]:
s = " N and N or N and N"
nouns = ["banana", "apple", "pear", "bread"]
power_set = gen_all_combos(nouns)
print(power_set)
v = []
for p in power_set[::-1]:
  resp = answer_qset(replace_nouns(s, nouns), delete_neg(p))
  print(resp)
  v.append(resp['res'][0])

[['banana', 'apple', 'pear', 'bread'], ['banana', 'apple', 'pear', 'not bread'], ['banana', 'apple', 'not pear', 'bread'], ['banana', 'apple', 'not pear', 'not bread'], ['banana', 'not apple', 'pear', 'bread'], ['banana', 'not apple', 'pear', 'not bread'], ['banana', 'not apple', 'not pear', 'bread'], ['banana', 'not apple', 'not pear', 'not bread'], ['not banana', 'apple', 'pear', 'bread'], ['not banana', 'apple', 'pear', 'not bread'], ['not banana', 'apple', 'not pear', 'bread'], ['not banana', 'apple', 'not pear', 'not bread'], ['not banana', 'not apple', 'pear', 'bread'], ['not banana', 'not apple', 'pear', 'not bread'], ['not banana', 'not apple', 'not pear', 'bread'], ['not banana', 'not apple', 'not pear', 'not bread']]
{'res': [0], 'q': 'Andre asked Boris: "Please buy me: bread and pear or apple and banana".  Boris bought nothing. Did Boris meet Andre\'s request? Oversatisfaction is allowed. Explain and write: 1 if "yes", 0 otherwise. You must write a number', 'a': '0'}
{'res':

In [267]:
print(v)
print("-")
dnf = find_vector("( N and N ) or ( N and N ) ", nouns)
cnf = find_vector("N and ( N or N ) and N ", nouns)
left = find_vector("( ( N and  N ) or N ) and N ", nouns)
right = find_vector(" N and  ( N  or  ( N  and N ) )  ", nouns)

print(dnf, cnf, left, right)

[0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1]
-
[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1] [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1] [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1]


In [268]:
print(coincidence(v, dnf))
print(coincidence(v, cnf))
print(coincidence(v, left))
print(coincidence(v, right))

0.8125
0.6875
0.6875
0.8125


In [269]:
def insert_logical_brackets(expr):
    expr = expr.replace("not N", "notN")
    elements = expr.split()
    if len(elements) < 3:
        expr = expr.replace("notN", "not N")
        return [expr]

    def recurse(elements):
        if len(elements) < 3:
            return [' '.join(elements).replace("notN", "not N")]

        results = set()
        for i in range(0, len(elements) - 2, 2):
            for j in range(i + 3, len(elements) + 1, 2):
                new_elements = elements[:i] + ['( ' + ' '.join(elements[i:j]) + ' )'] + elements[j:]
                results.add(' '.join(new_elements).replace("notN", "not N"))
                results.update(recurse(new_elements))

        return results

    return list(recurse(elements))


complex_expr = "N or N and not N or N and N"
combinations = insert_logical_brackets(complex_expr)
len(combinations)

89

In [215]:
from collections import defaultdict

def unique_parsings(ns, expr): #ns = nouns, expr - an expression w/o brackets
    combs = insert_logical_brackets(expr)
    res = list()
    for c in combs:
      res.append(str(find_vector(c, ns)))

    distinct = set(res)
    log_sent = list()
    dct = defaultdict(list)
    for c in combs:
        eval_ = str(find_vector(c, ns))
        dct[eval_] = c
        # dct[eval_].append(c)
        # if eval_ in distinct:
        #     log_sent.append(c)
        #     distinct.remove(eval_)
    return dct

ns = ["x", "y", "z", "w", "p"]
dct = unique_parsings(ns, complex_expr)
print(len(dct))
# for pair_ in dct.items():
#     print(pair_[0])

#     print(*pair_[1], sep = "\n")
#     print("-"*100)

10


In [202]:
ns = ["x", "y", "z", "w"]
print(find_vector("N and N or N and not N", ns))
print(find_vector("((( N ) and N ) or N ) and not N", ns))
print(find_vector(" N and ( N or ( N and not N ))", ns))
print(find_vector("N and ( ( N or  N ) and not N )", ns))
print(find_vector("( N and  ( N or  N ) ) and not N ", ns))

[0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1]
[0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1]
[0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]


In [318]:
def catalan(n):
    if n <= 1:
        return 1
    res = 0
    for i in range(n):
        res += catalan(i) * catalan(n-i-1)
    return res

# final test to json

In [376]:
#expressions to check
anon_exprs = gen_grammar_sequences(bool_expressions, d = 5)
print(anon_exprs)
len(anon_exprs)

['N or N', 'N and N or N and N or N', 'N and N and N or N and N or N', 'N and N and N', 'N', 'N and N or N or N', 'N or N and N', 'N and N', 'N and N and N or N or N', 'N and N or N and N', 'N or N or N', 'N and N or N', 'N and N and N or N', 'N or N and N or N', 'N and N and N or N and N']


15

In [ ]:
anon_exprs = anon_exprs
experiments_per_exprs = 15


for e in anon_exprs:
  analysis = dict()
  size = count_nouns(e)
  nouns = random.sample(fruits, size)
  base4 = {b[0]:(find_vector(b[1], nouns), b[1]) for b in four_parenthesize(e).items()} #{'dnf': ([0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1], 'N and N or N and N'), 'cnf': ([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1], ' ( N  ) and (  N or N  ) and (  N ) '), 'left': ([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1], ' (  (  ( N and N ) or N ) and N ) '), 'right': ([0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1], 'N and  ( N or  ( N and N )  ) ')}
  interpretation_space = unique_parsings(nouns, e) #defaultdict(<class 'list'>, {'[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1]': ['N and ( N or N ) and N', '( ( N and ( N or N ) ) and N )', '( N and ( N or N ) and N )', '( N and ( N or N ) ) and N', '( N and ( ( N or N ) and N ) )', 'N and ( ( N or N ) and N )'], '[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1]': ['( N and ( N or N and N ) )', 'N and ( N or ( N and N ) )', '( N and ( N or ( N and N ) ) )', 'N and ( N or N and N )'], '[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1]': ['( ( N and N ) or N and N )', '( N and N or N and N )', '( N and N ) or ( N and N )', '( ( N and N ) or ( N and N ) )', '( N and N ) or N and N', 'N and N or ( N and N )', '( N and N or ( N and N ) )'], '[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]': ['( N and N or N ) and N', '( ( ( N and N ) or N ) and N )', '( ( N and N or N ) and N )', '( ( N and N ) or N ) and N']})
  #####################################
  analysis['model'] = 'gpt-3.5'
  analysis["#_of_terms"] = size
  analysis["base_four"] = base4
  analysis["interpretation_space"] = interpretation_space
  analysis["upper_bound"] = catalan(size-1)
  analysis["experiments"] = dict()

  for i in range(experiments_per_exprs): #20 sets of nouns per expression
    print(e, i)
    analysis["experiments"][i] = dict()
    analysis["experiments"][i]["nouns"] = nouns
    analysis["experiments"][i]["response"] = []

    v_m = []
    power_set = gen_all_combos(nouns)
    for p in power_set[::-1]:
      resp = answer_qset(replace_nouns(e, nouns), delete_neg(p)) #{"res":res, "q": q, "a":a}

      analysis["experiments"][i]["response"].append(resp)

      v_m.append(resp['res'][0])

    analysis["experiments"][i]["model_vector"] = v_m

    is_results = dict()
    for v_i in list(interpretation_space.keys()):
      v_i = ast.literal_eval(v_i) #"[1, 0]" -> [1, 0]
      confusion = confusion_matrix(v_m, v_i) # model vs reality
      is_results[str(v_i)] = dict()
      is_results[str(v_i)]['accuracy'] = confusion['accuracy']
      is_results[str(v_i)]['f1'] = confusion['f1']
      is_results[str(v_i)]['precision'] = confusion['precision']
      is_results[str(v_i)]['recall'] = confusion['recall']
      is_results[str(v_i)]['cosine'] = cosine(v_i, v_m)
    four_base_results = dict()
    for v_i in base4:
      four_base_results[v_i] = dict()
      four_base_results[v_i]["interpretation"] = base4[v_i][1]
      four_base_results[v_i]["vector"] = base4[v_i][0]

      confusion = confusion_matrix(v_m, base4[v_i][0]) # model vs reality
      four_base_results[str(v_i)]['accuracy'] = confusion['accuracy']
      four_base_results[str(v_i)]['f1'] = confusion['f1']
      four_base_results[str(v_i)]['precision'] = confusion['precision']
      four_base_results[str(v_i)]['recall'] = confusion['recall']
      four_base_results[str(v_i)]['cosine'] = cosine(base4[v_i][0], v_m)
    analysis["experiments"][i]["is_results"] = is_results
    analysis["experiments"][i]["4_base_results"] = four_base_results
  with open("/content/drive/MyDrive/X_ACL/Boolean resolution/boolean-resolution/" + e +'.json', 'w', encoding='utf8') as outfile:
    json.dump(analysis, outfile, indent=4, separators=(',', ': '))

N or N 0
N or N 1
N or N 2
N or N 3
N or N 4
N or N 5
N or N 6
N or N 7
N or N 8
N or N 9
N or N 10
N or N 11
N or N 12
N or N 13
N or N 14
N and N or N and N or N 0
N and N or N and N or N 1
N and N or N and N or N 2
N and N or N and N or N 3
N and N or N and N or N 4
N and N or N and N or N 5
N and N or N and N or N 6
N and N or N and N or N 7
N and N or N and N or N 8
N and N or N and N or N 9
N and N or N and N or N 10
